## 필수과제 2

- 데이터를 가지고 아래 내용 정리해서 제출
  - 기존에 배웠던, OLS~ 변수에 대한 설명력 이해 등등
  - 오늘 배운 내용으로 최적의 모델링, alpha 값 찾기

In [56]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.formula.api import ols
import statsmodels.api as sm
from sklearn import datasets
import numpy as np

import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')

# 데이터 불러오기

In [44]:
# 데이터 불러오기
data = pd.read_csv("물류데이터.csv")
data = data[['송하인_격자공간고유번호', '수하인_격자공간고유번호',	'물품_카테고리',	'운송장_건수']]
data.head()

,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수
0,5011000595017300,2871000192069300,음반,3
1,4148000690043300,5011000264024400,문화컨텐츠,3
2,5011000078068400,1120000007005400,농산물,3
3,4127100048006400,5011000587019400,기타식품,7
4,5011000078068400,2823700010076300,농산물,3


In [54]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31684 entries, 0 to 31683
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   송하인_격자공간고유번호  31684 non-null  int64 
 1   수하인_격자공간고유번호  31684 non-null  int64 
 2   물품_카테고리       31684 non-null  object
 3   운송장_건수        31684 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 990.2+ KB


In [55]:
data.describe()

,송하인_격자공간고유번호,수하인_격자공간고유번호,운송장_건수
count,3.168400e+04,3.168400e+04,31684.000000
mean,4.648814e+15,3.801369e+15,4.879308
std,8.341336e+14,1.314660e+15,6.756473
min,1.111000e+15,1.111000e+15,3.000000
25%,4.713000e+15,2.826000e+15,3.000000
50%,5.011000e+15,4.159000e+15,3.000000
75%,5.013001e+15,5.011000e+15,5.000000
max,5.013001e+15,5.013001e+15,413.000000


# 데이터 전처리

## 1.물품_카테고리 LabelEncoding
- 물품_카테고리 데이터가 범주형 데이터로 이루어져 있음
- LabelEncoding을 통해 변환

In [346]:
data2 = data.copy()
data2['물품_카테고리'].unique()

array(['음반', '문화컨텐츠', '농산물', '기타식품', '수산', '아우터', '신발', '건강식품', '음료',
       '가공식품', '스킨케어', '축산', '헤어케어', '기저귀/물티슈', '의료용품', '기타디지털/가전', '상의',
       '건강용품', '바디케어', '기타스포츠/레저', '출산/유아동의류', '반려동물', '완구/매트', '과자',
       '생활용품', '하의', '문구/사무용품', '선케어', '다이어트식품', '기타패션의류', '냉동/간편조리식품',
       '네일케어', '서재/사무용가구', '선글라스/안경테', '기타화장품/미용', '등산', '모니터', '주방용품',
       '색조메이크업', '침구세트', '침구단품', '인테리어소품', '양말/스타킹', '클렌징', '공구', '이미용가전',
       'PC', '커튼/블라인드', '헤어스타일링', '주방가구', 'DIY자재/용품', '헤어액세서리', '김치',
       '위생/건강용품', '침실가구', '가방', '주방가전', '낚시', '재활운동용품', '자동차용품', '뷰티소품',
       '주얼리', '골프', 'PC주변기기', '수납/정리용품', '음향가전', '잠옷', '패션소품', '언더웨어',
       '원피스/점프슈트', '스포츠잡화', '수납가구', '취미용품', '아웃도어가구', '계절가전', '홈데코', '캠핑',
       '남성화장품', '분유/이유식/아기간식', '베이스메이크업', '헬스', '향수', '모자', '카페트/러그',
       '구강위생용품', '눈관리용품', '기타출산/육아', '기타패션잡화', '기능성', '반찬', '욕실용품',
       '스마트디바이스액세서리', '게임기/타이틀', '생활가전', '유아가구', '출산/유아동잡화', '세탁용품',
       '태블릿PC/노트북액세서리', '스마트디바이스', '지갑'], dtype=object)

In [347]:
# 범주형 변수 숫자 변환
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
data2['물품_카테고리'] = le.fit_transform(data2['물품_카테고리'])
data2.head()

,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수
0,5011000595017300,2871000192069300,67,3
1,4148000690043300,5011000264024400,34,3
2,5011000078068400,1120000007005400,27,3
3,4127100048006400,5011000587019400,17,7
4,5011000078068400,2823700010076300,27,3


## 2.격자공간고유번호

In [348]:
print('송하인 격자공간고유번호 개수 :',len(data2['송하인_격자공간고유번호'].unique()))
print('수하인 격자공간고유번호 개수 :',len(data2['수하인_격자공간고유번호'].unique()))

송하인 격자공간고유번호 개수 : 4229
수하인 격자공간고유번호 개수 : 26875


In [349]:
def split(data, range1, range2):
  result = []
  for x in data:
    result.append(str(x)[range1:range2])
  return result

In [350]:
# 송하인 격자공간고유번호
data2['송하인_격자공간고유번호15'] = split(data2['송하인_격자공간고유번호'],0,5)
# 수하인 격자공간고유번호
data2['수하인_격자공간고유번호15'] = split(data2['수하인_격자공간고유번호'],0,5)
data2

,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수,송하인_격자공간고유번호15,수하인_격자공간고유번호15
0,5011000595017300,2871000192069300,67,3,50110,28710
1,4148000690043300,5011000264024400,34,3,41480,50110
2,5011000078068400,1120000007005400,27,3,50110,11200
3,4127100048006400,5011000587019400,17,7,41271,50110
4,5011000078068400,2823700010076300,27,3,50110,28237
...,...,...,...,...,...,...
31679,4471000290087200,5011000213073200,55,3,44710,50110
31680,1129000014045300,5011000319087100,52,4,11290,50110
31681,1129000014045300,5011000263065200,52,6,11290,50110
31682,4127300065073100,5011000264061200,79,7,41273,50110


In [351]:
print('송하인 격자공간고유번호15 개수 :',len(data2['송하인_격자공간고유번호15'].unique()))
print('수하인 격자공간고유번호 개수15 :',len(data2['수하인_격자공간고유번호15'].unique()))

송하인 격자공간고유번호15 개수 : 237
수하인 격자공간고유번호 개수15 : 251


In [352]:
data2['송하인_격자공간고유번호15'] = le.fit_transform(data2['송하인_격자공간고유번호15'])
data2['수하인_격자공간고유번호15'] = le.fit_transform(data2['수하인_격자공간고유번호15'])

In [353]:
data2['송하인_격자공간고유번호'] = le.fit_transform(data2['송하인_격자공간고유번호'])
data2['수하인_격자공간고유번호'] = le.fit_transform(data2['수하인_격자공간고유번호'])
data2.head()

,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수,송하인_격자공간고유번호15,수하인_격자공간고유번호15
0,3296,7435,67,3,235,57
1,1474,22754,34,3,105,249
2,2398,413,27,3,235,3
3,1071,25128,17,7,89,249
4,2398,6948,27,3,235,54


## 3.데이터 스케일링

In [354]:
columns = data2.columns

In [355]:
from sklearn.preprocessing import StandardScaler
st = StandardScaler()
st.fit(data2)
data2 = st.transform(data2)

In [356]:
data2=pd.DataFrame(data2, columns=columns)
data2

,송하인_격자공간고유번호,수하인_격자공간고유번호,물품_카테고리,운송장_건수,송하인_격자공간고유번호15,수하인_격자공간고유번호15
0,0.539663,-0.851994,2.082329,-0.278154,0.546494,-0.873726
1,-1.178825,1.054671,0.129568,-0.278154,-1.386810,1.283671
2,-0.307320,-1.725981,-0.284654,-0.278154,0.546494,-1.480494
3,-1.558930,1.350148,-0.876399,0.313881,-1.624755,1.283671
4,-0.307320,-0.912608,-0.284654,-0.278154,0.546494,-0.907435
...,...,...,...,...,...,...
31679,-0.689311,0.989078,1.372234,-0.278154,-0.672975,1.283671
31680,-2.424776,1.177517,1.194710,-0.130145,-2.844224,1.283671
31681,-2.424776,1.049070,1.194710,0.165872,-2.844224,1.283671
31682,-1.549498,1.059649,2.792423,0.313881,-1.609884,1.283671


# 데이터 분할

In [357]:
# 데이터 분할
X = data2.drop(['운송장_건수'], axis = 1)
y = data2['운송장_건수']
X_train, X_test , y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=111)

# 변수확인

## 1.OLS

In [358]:
from statsmodels.formula.api import ols
import statsmodels.api as sm

In [359]:
cols = ['송하인_격자공간고유번호', '수하인_격자공간고유번호', '물품_카테고리', '송하인_격자공간고유번호15', '수하인_격자공간고유번호15']
model=sm.OLS.from_formula('운송장_건수 ~ '+'+'.join(cols), data=data2)
res = model.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 운송장_건수   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     44.44
Date:                Sat, 15 Jul 2023   Prob (F-statistic):           7.34e-46
Time:                        09:45:13   Log-Likelihood:                -44847.
No. Observations:               31684   AIC:                         8.971e+04
Df Residuals:                   31678   BIC:                         8.976e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept       7.698e-17      0.006   1.37e-14      1.000      -0.011       0.011
송하인_격자공간고유번호       0.0187      0.012      1.624      0.104      -0.004       0.041
수하인_격자공간고유번호      -0.1146      0.033     -3.509      0.000      -0.179      -0.051
물품_카테고리           -0.0010      0.006     -0.173      0.862      -0.012       0.010
송하인_격자공간고유번호15    -0.0357      0.013     -2.787      0.005      -0.061      -0.011
수하인_격자공간고유번호15     0.1782      0.034      5.236      0.000       0.111       0.245
==============================================================================
Omnibus:                    66776.798   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        556690587.381
Skew:                          18.074   Prob(JB):                         0.00
Kurtosis:                     651.364   Cond. No.                         15.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [362]:
# p-value 큰 변수 제거 후 다시 비교
cols = [ '수하인_격자공간고유번호', '송하인_격자공간고유번호15', '수하인_격자공간고유번호15']
model=sm.OLS.from_formula('운송장_건수 ~ '+'+'.join(cols), data=data2)
res = model.fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 운송장_건수   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     73.17
Date:                Sat, 15 Jul 2023   Prob (F-statistic):           3.75e-47
Time:                        09:47:11   Log-Likelihood:                -44848.
No. Observations:               31684   AIC:                         8.970e+04
Df Residuals:                   31680   BIC:                         8.974e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept       7.698e-17      0.006   1.37e-14      1.000      -0.011       0.011
수하인_격자공간고유번호      -0.1146      0.033     -3.510      0.000      -0.179      -0.051
송하인_격자공간고유번호15    -0.0194      0.008     -2.404      0.016      -0.035      -0.004
수하인_격자공간고유번호15     0.1778      0.034      5.226      0.000       0.111       0.245
==============================================================================
Omnibus:                    66772.475   Durbin-Watson:                   2.010
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        556435700.168
Skew:                          18.071   Prob(JB):                         0.00
Kurtosis:                     651.215   Cond. No.                         13.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

- 독립성은 Durbin-Watson 1.5~2.5 사이이면 독립으로 판단하고 회귀모형을 적합하다고 하는데, 2.010으로 독립성이 성립한다고 할 수 있다
- 또한 p-value가 큰 변수들을 삭제 전후의 큰 차이가 없어 제거하지 않고 진행한다

## 2.VIF 확인

In [360]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif = pd.DataFrame()
vif['VIF factor'] = [variance_inflation_factor(data2, i) for i in range(data2.shape[1])]
vif['features'] = data2.columns
vif = vif.sort_values('VIF factor').reset_index(drop=True)
vif

,VIF factor,features
0,1.007014,운송장_건수
1,1.034951,물품_카테고리
2,4.239974,송하인_격자공간고유번호
3,5.240833,송하인_격자공간고유번호15
4,34.024159,수하인_격자공간고유번호
5,36.972817,수하인_격자공간고유번호15


- vif가 10보다 큰 변수들이 두개가 존재한다(수하인_격자공간고유번호
, 수하인_격자공간고유번호15)

# 다항회귀

In [314]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression

## 1.단순선형회귀

In [333]:
lr = LinearRegression()

In [334]:
# 단순회귀식 계산
lr.fit(X_train,y_train)
y_lin_fit = lr.predict(X_test)
l_r2 = r2_score(y_test, y_lin_fit)

print("R2 score :", l_r2)

R2 score : 0.006953551792119095


## 2.2차 다항회귀

In [335]:
quadrastic = PolynomialFeatures(degree=2) # 2차 다항회귀
X_quad = quadrastic.fit_transform(X_train)    # 데이터를 2차 다항회귀로 변환

In [336]:
lr.fit(X_quad,y_train)
y_quad_fit = lr.predict(quadrastic.fit_transform(X_test))
q_r2 = r2_score(y_test, y_quad_fit)
print("2차 다항회귀 R^2 :", q_r2)

2차 다항회귀 R^2 : 0.006455476000493676


## 3.3차 다항회귀

In [337]:
cubic = PolynomialFeatures(degree=3)      # 3차 다항회귀
X_cubic = cubic.fit_transform(X_train)        # 데이터를 3차 다항회귀로 변환

In [338]:
# 3차 다항회귀 모델
lr.fit(X_cubic,y_train)
y_cubic_fit = lr.predict(cubic.fit_transform(X_test))
c_r2 = r2_score(y_test, y_cubic_fit)
print("3차 다항회귀 R^2 :", c_r2)

3차 다항회귀 R^2 : 0.002968805657620699


## 4.결과
- 단순선형에서 다항으로 늘어날수록 R^2값이 작아짐
- 단순선형이 가장 좋은 것으로 보임

# 규제

In [303]:
from sklearn.linear_model import Lasso, ElasticNet

# alpha값 따른 회귀 모델의 폴드 평균 RMSE를 출력하고 회귀 계수값들을 DF 변환
def get_linear_reg_eval(model_name, params = None, X_data_n =None, y_target_n=None,
                       verbose= True, return_coeff=True):
    coeff_df =pd.DataFrame()
    if verbose : print('######', model_name,'#######')
    for param in params:
        if model_name =='Ridge' : model = Ridge(alpha=param)
        elif model_name =='Lasso' : model = Lasso(alpha=param)
        elif model_name =='ElasticNet' : model = ElasticNet(alpha=param, l1_ratio=0.7)
        neg_mse_scores = cross_val_score(model, X_data_n, y_target_n,
                                         scoring= 'neg_mean_squared_error',cv= 5)
        avg_rmse = np.mean(np.sqrt(-1 * neg_mse_scores))
        print('alpha {0} 일 때 5 폴드 세트의 평균 RMSE : {1}'.format(param, avg_rmse))
        #cross_val_score 는 evaluation metric만 반환하므로 모델을 다시 학습하여 회귀 계수 추출

        model.fit(X_data_n, y_target_n)
        if return_coeff:
            #alpha에 따른 피처별 회귀 계수를 Series로 변환하고 이를 DataFrame의 컬럼 추가
            coeff = pd.Series(data=model.coef_, index =X_data_n.columns)
            colname = 'alpha:'+str(param)
            coeff_df[colname] = coeff
    return coeff_df

## 1.Lasso

In [309]:
# alpha
alphas = [0, 0.001, 0.1, 1, 100]

coeff_lasso_df = get_linear_reg_eval('Lasso', params= alphas, X_data_n =X_train, y_target_n= y_train)
coeff_df.sort_values(by=sort_columns, ascending=False)

###### Lasso #######
alpha 0 일 때 5 폴드 세트의 평균 RMSE : 1.0111777871306862
alpha 0.001 일 때 5 폴드 세트의 평균 RMSE : 1.0112238526040052
alpha 0.1 일 때 5 폴드 세트의 평균 RMSE : 1.014500915688966
alpha 1 일 때 5 폴드 세트의 평균 RMSE : 1.014500915688966
alpha 100 일 때 5 폴드 세트의 평균 RMSE : 1.014500915688966


,alpha:0,alpha:0.0001,alpha:0.001,alpha:0.1,alpha:1
수하인_격자공간고유번호1~5,0.185348,0.185348,0.185347,0.185305,0.184915
송하인_격자공간고유번호,0.014054,0.014054,0.014054,0.014053,0.014046
물품_카테고리,-0.009991,-0.009991,-0.009991,-0.009991,-0.009990
송하인_격자공간고유번호1~5,-0.036220,-0.036220,-0.036220,-0.036222,-0.036237
수하인_격자공간고유번호,-0.123583,-0.123583,-0.123583,-0.123542,-0.123169


## 2.Ridge

In [310]:
# 각 alpha에 따른 회귀 계수 값을 데이터로 저장하는 데이터 프레임
coeff_df = pd.DataFrame()

for pos, alpha in enumerate(alphas):
    ridge = Ridge(alpha= alpha)
    ridge.fit(X_train, y_train)
    coeff = pd.Series(data=ridge.coef_, index=X_train.columns)
    colname = 'alpha:'+str(alpha)
    coeff_df[colname]= coeff
    coeff = coeff.sort_values(ascending=False)

In [311]:
# Ridge
coeff_Ridge_df = get_linear_reg_eval('Ridge',params= alphas, X_data_n =X_train, y_target_n= y_train)

###### Ridge #######
alpha 0 일 때 5 폴드 세트의 평균 RMSE : 1.0111777871306862
alpha 0.001 일 때 5 폴드 세트의 평균 RMSE : 1.0111777866713099
alpha 0.1 일 때 5 폴드 세트의 평균 RMSE : 1.0111777412507021
alpha 1 일 때 5 폴드 세트의 평균 RMSE : 1.0111773335387209
alpha 100 일 때 5 폴드 세트의 평균 RMSE : 1.0111669866590949


In [312]:
ridge_alphas = [0, 0.1,1,10,100]
sort_columns = 'alpha:'+ str(ridge_alphas[0])
coeff_df.sort_values(by=sort_columns, ascending=False)

,alpha:0,alpha:0.001,alpha:0.1,alpha:1,alpha:100
수하인_격자공간고유번호1~5,0.185348,0.185347,0.185305,0.184915,0.151346
송하인_격자공간고유번호,0.014054,0.014054,0.014053,0.014046,0.013269
물품_카테고리,-0.009991,-0.009991,-0.009991,-0.009990,-0.009927
송하인_격자공간고유번호1~5,-0.036220,-0.036220,-0.036222,-0.036237,-0.037382
수하인_격자공간고유번호,-0.123583,-0.123583,-0.123542,-0.123169,-0.091017


## 3.ElasticNet

In [313]:
# ElasticNet
alphas = [0.001, 0.1, 1, 10]
coeff_ElasticNet_df = get_linear_reg_eval('ElasticNet',params= alphas, X_data_n =X_train, y_target_n= y_train)
coeff_df.sort_values(by=sort_columns, ascending=False)

###### ElasticNet #######
alpha 0.001 일 때 5 폴드 세트의 평균 RMSE : 1.011197929902939
alpha 0.1 일 때 5 폴드 세트의 평균 RMSE : 1.013831002809687
alpha 1 일 때 5 폴드 세트의 평균 RMSE : 1.014500915688966
alpha 10 일 때 5 폴드 세트의 평균 RMSE : 1.014500915688966


,alpha:0,alpha:0.001,alpha:0.1,alpha:1,alpha:100
수하인_격자공간고유번호1~5,0.185348,0.185347,0.185305,0.184915,0.151346
송하인_격자공간고유번호,0.014054,0.014054,0.014053,0.014046,0.013269
물품_카테고리,-0.009991,-0.009991,-0.009991,-0.009990,-0.009927
송하인_격자공간고유번호1~5,-0.036220,-0.036220,-0.036222,-0.036237,-0.037382
수하인_격자공간고유번호,-0.123583,-0.123583,-0.123542,-0.123169,-0.091017
